# Inference

In [14]:
from PIL import Image
import requests
from transformers import AutoProcessor, Pix2StructForConditionalGeneration
import torch

In [17]:
processor = AutoProcessor.from_pretrained("google/pix2struct-textcaps-base")
model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-textcaps-base").to('cuda')

url = "https://cdn.discordapp.com/attachments/1022151606604660827/1284574588914896906/51oHGvSvuL.jpg?ex=66e720d0&is=66e5cf50&hm=bf80829aee3b5b7722b6853c5fb0459378538fb72eeb07c35016b95432286f90&"
image = Image.open(requests.get(url, stream=True).raw)

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
text = 'length is'
inputs = processor(text=text, images=image, return_tensors="pt", add_special_tokens=False).to('cuda')
generated_ids = model.generate(**inputs, max_new_tokens=5)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

In [20]:
generated_ids = model.generate(**inputs, max_new_tokens=5)

In [21]:
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

length is 40 inches


# Fine tuning

In [ ]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFace

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [1]:
hyperparameters = {
    'model_name_or_path':'google/pix2struct-base',
    'output_dir':'/opt/ml/model'
    # add your remaining hyperparameters
    # more info here https://github.com/huggingface/transformers/tree/v4.37.0/path/to/script
}

# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.37.0'}

In [22]:
# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./path/to/script',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    git_config=git_config,
    transformers_version='4.37.0',
    pytorch_version='2.1.0',
    py_version='py310',
    hyperparameters = hyperparameters
)

# starting the train job
huggingface_estimator.fit()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/arjun/.config/sagemaker/config.yaml


NoCredentialsError: Unable to locate credentials